In [1]:
from bs4 import BeautifulSoup as bs
from itertools import cycle
from torrequest import TorRequest
import random
import collections
from random import choice
import time
import requests
import pandas as pd
import re
import os

In [2]:
def get_user_agent():
    
    ua_list = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
               "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",
               "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",
               "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",
               "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",
               "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",
               "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"
               ]
    
    return random.choice(ua_list)

In [3]:
def get_proxies():
    
    proxy_url = 'https://github.com/TheSpeedX/PROXY-List/blob/master/http.txt'

    response = requests.get(proxy_url)
    soup = bs(response.content, 'lxml').find_all('td',{'class':'blob-code blob-code-inner js-file-line'})
    proxies = ["http://"+i.text for i in soup]

    keys = []
    for i in range(len(proxies)):
        keys.append('http')

    proxy_list = [{L2 : L1} for (L1,L2) in zip(proxies, keys)]  
    #print(proxy_list)
    return random.choice(proxy_list)

In [6]:
test_url = 'https://www.google.com/'
tech_url = 'https://www.techpowerup.com/gpu-specs/'

In [17]:
def parse_html(url):
    
    header = {'User-Agent': get_user_agent()}
    proxy = get_proxies()
    
    try:
        #time.sleep(random.randint(45,700))
        #time.sleep(random.randint(150,400))
        response = requests.get(url, headers= header, proxies= proxy, timeout= 3)
        status_code = response.status_code
        content = response.content
        soup = bs(content, "lxml")
        print(header)
        print(proxy)
        
        return soup, status_code
    
    except Exception as e:
        print(f"Error: {e}")
    
        
        #print(header)
        #print(proxy)
    

In [11]:
soup, status_code = parse_html(tech_url)

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25'}
{'http': 'http://144.91.89.245:3128'}


In [ ]:
print(soup)

In [8]:
gpu_links = []

for i in soup.find('table', class_="processors").find_all('tr'):
    next_page = i.find('a')
    if next_page != None:
        page_list = next_page.attrs['href']
        gpu_links.append(tech_url.strip('/gpu-spec/')+page_list)



In [18]:
col_label_list = []
data_list = []
temp_dict = {}

for idx, i in enumerate(gpu_links):
    print(f"-----{i}-------")
    time.sleep(random.randint(45,700))
    soup, status_code = parse_html(gpu_links[idx])
    print(status_code)
    if status_code != 200:
        soup, status_code = parse_html(gpu_links[idx])
        print(status_code)
   
    title = soup.find('h1', class_="gpudb-name").text
    #title_list.append(title)
   
    gpu_stats = soup.find('div').find_all('dl')
   
    temp_dict[title] = {}
    for jdx, j in enumerate(gpu_stats):
        column_label = j.find('dt').text
        data = j.find('dd').text.replace('\n',"").replace('\t',"")
        #col_label_list.append(column_label)
        #data_list.append(data)
        temp_dict[title][column_label] = data
        
    for key, value in zip(column_label, data):
        temp_dict[title][column_label] = data       
       

-----https://www.techpowerup.com/gpu-specs/geforce-rtx-3060.c3682-------
{'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0'}
{'http': 'http://45.226.185.2:8080'}
200
-----https://www.techpowerup.com/gpu-specs/geforce-rtx-3060-ti.c3681-------
{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10'}
{'http': 'http://41.65.236.56:1981'}
200
-----https://www.techpowerup.com/gpu-specs/geforce-rtx-3090-ti.c3829-------
{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36'}
{'http': 'http://187.120.242.177:3629'}
200
-----https://www.techpowerup.com/gpu-specs/geforce-rtx-3080.c3621-------
{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10'}
{'http': 'http://170.84.50.225:4153'}
200
-----https://www.techpowerup.com/

In [294]:
back_updict = temp_dict

In [295]:
print(back_updict)

{'NVIDIA GeForce RTX 3060': {'Graphics Processor': 'GA106', 'GPU Name': 'GA106', 'GPU Variant': 'GA106-300-A1', 'Architecture': 'Ampere', 'Foundry': 'Samsung', 'Process Size': '8 nm', 'Transistors': '12,000 million', 'Die Size': '276 mm²', 'Release Date': 'Jan 12th, 2021', 'Availability': 'Feb 25th, 2021', 'Generation': 'GeForce 30', 'Predecessor': 'GeForce 20', 'Successor': 'GeForce 40', 'Production': 'Active', 'Launch Price': '329 USD', 'Bus Interface': 'PCIe 4.0 x16', 'Reviews': '60 in our database ', 'Base Clock': '1320 MHz', 'Boost Clock': '1777 MHz', 'Memory Clock': '1875 MHz15 Gbps effective', 'Memory Size': '12 GB', 'Memory Type': 'GDDR6', 'Memory Bus': '192 bit', 'Bandwidth': '360.0 GB/s', 'Shading Units': '3584', 'TMUs': '112', 'ROPs': '48', 'SM Count': '28', 'Tensor Cores': '112', 'RT Cores': '28', 'L1 Cache': '128 KB (per SM)', 'L2 Cache': '3 MB', 'Pixel Rate': '85.30 GPixel/s', 'Texture Rate': '199.0 GTexel/s', 'FP16 (half) performance': '12.74 TFLOPS (1:1)', 'FP32 (float)

In [312]:
def dict_to_df(my_dict):

    df_temp = pd.DataFrame(columns=['Name','SKU','Process(nm)','Die(mm²)','Date','MSRP(USD)','Bus','BaseClock(MHz)','BoostClock(MHz)'
                                   ,'MemoryClock(MHz)','MemorySize(GB)','MemBus(bit)','Bandwidth(GB/s)','ShadingUnits','RTCores'
                                   ,'PixelRate(GPixel/s)','TextureRate(GTexel/s)','FP16','FP32','FP64','TDP'])

    for key, val in my_dict.items():

        gpu_name = key
        if 'Board Number' in val:
            sku = val['Board Number']
        proc = val['Process Size'].strip('nm')
        die_size = val['Die Size'].strip('mm²')
        rel_date = val['Release Date'].replace('th,','').replace('st,','').replace('nd,','').replace('rd,','')
        if 'Launch Price' in val:
            msrp = val['Launch Price'].strip('USD').split(' ')[0].replace(',','')
        bus_interface = val['Bus Interface']
        if 'Base Clock' in val:
            base_clock = val['Base Clock'].strip('MHz')
        if 'Boost Clock' in val:
            boost_clock = val['Boost Clock'].strip('MHz')
        mem_clock = val['Memory Clock'].split(' ')[0]
        mem_size = val['Memory Size'].strip('GB').strip('M')
        mem_type = val['Memory Type']
        mem_bus = val['Memory Bus'].strip('bit')
        band_wid = val['Bandwidth'].strip('GB/s').replace(',','')
        shade_unit = val['Shading Units']
        if 'RT Cores' in val:
            rt_cores = val['RT Cores']
        pix_rate = val['Pixel Rate'].strip('GPixel/s')
        tex_rate = val['Texture Rate'].strip('GTexel/s').replace(',','')
        if 'FP16 (half) performance' in val:
            fp16 = val['FP16 (half) performance'].split(' ')[0].replace(',','')
        fp32 = val['FP32 (float) performance'].split(' ')[0].replace(',','')
        if 'FP64 (double) performance' in val:
            fp64 = val['FP64 (double) performance'].split(' ')[0].replace(',','')
        tdp = val['TDP'].strip('W')

        df_temp = df_temp.append({'Name':gpu_name,'SKU':sku,'Process(nm)':proc,'Die(mm²)':die_size,'Date':rel_date,'MSRP(USD)':msrp
                                  ,'Bus':bus_interface,'BaseClock(MHz)':base_clock,'BoostClock(MHz)':boost_clock,'MemoryClock(MHz)':mem_clock
                                  ,'MemorySize(GB)':mem_size,'MemBus(bit)': mem_bus,'Bandwidth(GB/s)':band_wid,'ShadingUnits':shade_unit
                                  ,'RTCores':rt_cores,'PixelRate(GPixel/s)':pix_rate,'TextureRate(GTexel/s)':tex_rate,'FP16':fp16,'FP32':fp32
                                  ,'FP64':fp64,'TDP':tdp}, ignore_index = True)

        df_temp['Date'] = pd.to_datetime(df_temp['Date'])
        df = df_temp.astype({'Process(nm)':'int64','Die(mm²)':'int64','MSRP(USD)':'int64','BaseClock(MHz)':'int64','BoostClock(MHz)':'int64'
                             ,'MemoryClock(MHz)':'int64','MemorySize(GB)':'int64','MemBus(bit)':'int64','Bandwidth(GB/s)':'float64','ShadingUnits':'int64'
                             ,'RTCores':'int64','PixelRate(GPixel/s)':'float64','TextureRate(GTexel/s)':'float64','FP16':'float64','FP32':'float64'
                             ,'FP64':'float64','TDP':'int64'}) 

    df.to_csv('GPU_Specs.csv')

    return df

In [313]:
df =  dict_to_df(back_updict)

In [315]:
df.head()

,Name,SKU,Process(nm),Die(mm²),Date,MSRP(USD),Bus,BaseClock(MHz),BoostClock(MHz),MemoryClock(MHz),...,MemBus(bit),Bandwidth(GB/s),ShadingUnits,RTCores,PixelRate(GPixel/s),TextureRate(GTexel/s),FP16,FP32,FP64,TDP
0,NVIDIA GeForce RTX 3060,PG190 SKU 60,8,276,2021-01-12,329,PCIe 4.0 x16,1320,1777,1875,...,192,360.0,3584,28,85.3,199.0,12.74,12.74,199.0,170
1,NVIDIA GeForce RTX 3060 Ti,PG142 SKU 20,8,392,2020-12-01,399,PCIe 4.0 x16,1410,1665,1750,...,256,448.0,4864,38,133.2,253.1,16.20,16.20,253.1,200
2,NVIDIA GeForce RTX 3090 Ti,PG136 SKU 5,8,628,2022-01-27,1999,PCIe 4.0 x16,1560,1860,1313,...,384,1008.0,10752,84,208.3,625.0,40.00,40.00,625.0,450
3,NVIDIA GeForce RTX 3080,PG132 SKU 30,8,628,2020-09-01,699,PCIe 4.0 x16,1440,1710,1188,...,320,760.3,8704,68,164.2,465.1,29.77,29.77,465.1,320
4,AMD Radeon RX 6600 XT,PG132 SKU 30,7,237,2021-07-30,379,PCIe 4.0 x8,1968,2589,2000,...,128,256.0,2048,32,165.7,331.4,21.21,10.60,662.8,160


In [242]:
df_temp.head()

,Name,SKU,Process(nm),Die(mm²),Date,MSRP(USD),Bus,BaseClock(MHz),BoostClock(MHz),MemoryClock(MHz),...,MemBus(bit),Bandwidth(GB/s),ShadingUnits,RTCores,PixelRate(GPixel/s),TextureRate(GTexel/s),FP16,FP32,FP64,TDP
0,NVIDIA GeForce RTX 3060,PG190 SKU 60,8,276,2021-01-12,329,PCIe 4.0 x16,1320,1777,1875,...,192,360.0,3584,28,85.30,199.0,12.74,12.74,199.0,170
1,NVIDIA GeForce RTX 3060 Ti,PG142 SKU 20,8,392,2020-12-01,399,PCIe 4.0 x16,1410,1665,1750,...,256,448.0,4864,38,133.2,253.1,16.20,16.20,253.1,200
2,NVIDIA GeForce RTX 3090 Ti,PG136 SKU 5,8,628,2022-01-27,1999,PCIe 4.0 x16,1560,1860,1313,...,384,"1,008",10752,84,208.3,625.0,40.00,40.00,625.0,450
3,NVIDIA GeForce RTX 3080,PG132 SKU 30,8,628,2020-09-01,699,PCIe 4.0 x16,1440,1710,1188,...,320,760.3,8704,68,164.2,465.1,29.77,29.77,465.1,320
4,AMD Radeon RX 6600 XT,PG132 SKU 30,7,237,2021-07-30,379,PCIe 4.0 x8,1968,2589,2000,...,128,256.0,2048,32,165.7,331.4,21.21,10.60,662.8,160


In [232]:
df_temp['MSRP(USD)']

0      329
1      399
2     1999
3      699
4      379
5      249
6      159
7      499
8      599
9      329
10     139
11     349
12     149
13    1499
14    1199
15     479
16     299
17     199
18     329
19     699
20     999
21     199
22     379
23     229
24    1099
25     169
26     149
27      79
28     599
29     229
30     999
31      79
32     279
33     279
34     109
35     109
36     159
37     649
38     649
39     149
40     149
41     219
42      99
43     159
44     159
45     499
46     499
47     249
48     549
49     499
Name: MSRP(USD), dtype: object

In [ ]:
 green = df.loc[df['Name'].str.contains("nvidia", case=False)]
    

In [215]:
df_temp.Date.head()

0   2021-01-12
1   2020-12-01
2   2022-01-27
3   2020-09-01
4   2021-07-30
Name: Date, dtype: datetime64[ns]